##Install Dependencies

In [27]:
!pip install sentencepiece
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./
%cd /content

!pip install fairseq

!pip install wandb
# os.getcwd()
#If Something goes wrong with the bleu score generation
# !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

fatal: destination path 'fairseq' already exists and is not an empty directory.
/content/fairseq/fairseq
Obtaining file:///content/fairseq/fairseq
ERROR: file:///content/fairseq/fairseq does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
/content
  Using cached torch-2.1.1-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.

##Imports

In [28]:
import os
import sentencepiece as spm
import wandb
wandb.login(key = "46cb1e2ffa78177f23adbe2d7d16cf09a6176348")
from google.colab import drive
drive.mount('/content/drive')

wandb: Currently logged in as: arabic-swahilimt (arabic-swahili). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Preparing File Directories

In [30]:
!mkdir /content/bpe_dict_path
%mkdir /content/drive/MyDrive/logs
!mkdir /content/train/
!mkdir /content/test/
!mkdir /content/val/
!mkdir /content/train_bpe/
!mkdir /content/test_bpe/
!mkdir /content/val_bpe/

mkdir: cannot create directory ‘/content/bpe_dict_path’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/logs’: File exists
mkdir: cannot create directory ‘/content/train/’: File exists
mkdir: cannot create directory ‘/content/test/’: File exists
mkdir: cannot create directory ‘/content/val/’: File exists
mkdir: cannot create directory ‘/content/train_bpe/’: File exists
mkdir: cannot create directory ‘/content/test_bpe/’: File exists
mkdir: cannot create directory ‘/content/val_bpe/’: File exists


##Data Preprocessing

In [ ]:
#Import The data from this working Directory
!unzip /{PATH_TO_THE_DATA_FILE}/txt_splitted_files.zip
!unzip /{PATH_TO_THE_DATA_FILE}/DATA.zip

In [6]:
#the entire files -used to train the bpe models
All_ar_data = open("/content/Arabic.txt", "r").readlines()
All_sw_data = open("/content/Swahili.txt","r").readlines()
# the standard training files -from it will emerge the validation set
train_ar_data = open("/content/train.ar", "r").readlines()
train_sw_data = open("/content/train.sw", "r").readlines()
# the standard test files
test_ar_data = open("/content/test.ar", "r").readlines()
test_sw_data = open("/content/test.sw", "r").readlines()

#Creating the final training set
with open("/content/train/train.ar", "w") as fb:
    fb.writelines(train_ar_data[2000:])
with open("/content/train/train.sw", "w") as fb:
    fb.writelines(train_sw_data[2000:])

#creating a validation set from the standard training set
with open("/content/val/val.ar", "w") as fb:
    fb.writelines(train_ar_data[:2000])
with open("/content/val/val.sw", "w") as fb:
    fb.writelines(train_sw_data[:2000])

#the Standard test set
with open("/content/test/test.ar", "w") as fb:
    fb.writelines(test_ar_data)
with open("/content/test/test.sw", "w") as fb:
    fb.writelines(test_sw_data)

In [35]:
#checking the splits
train_sw_data = open("/content/train/train.sw", "r").readlines()
test_sw_data = open("/content/test/test.sw", "r").readlines()
val_sw_data = open("/content/val/val.sw", "r").readlines()
print(len(train_sw_data) , len(test_sw_data) ,len(val_sw_data))

train_ar_data = open("/content/train/train.ar", "r").readlines()
test_ar_data = open("/content/test/test.ar", "r").readlines()
val_ar_data = open("/content/val/val.ar", "r").readlines()
print(len(train_ar_data) , len(test_ar_data) ,len(val_ar_data))

27531 3281 2000
27531 3281 2000


In [36]:
#Training the BPE Models

ar_file_path = "/content/Arabic.txt"
#Arabic training file "all arabic text"
sw_file_path = "/content/Swahili.txt"
#swalihi_training file



dict_path = "/content/bpe_dict_path"
#dictionary "create directory and name it as you like"
ar_train_input = "/content/train/train.ar"
ar_train_bpe_output = "/content/train_bpe/train.bpe.ar"
sw_train_input = "/content/train/train.sw"
sw_train_bpe_output = "/content/train_bpe/train.bpe.sw"

ar_val_input = "/content/val/val.ar"
ar_val_bpe_output = "/content/val_bpe/val.bpe.ar"
sw_val_input = "/content/val/val.sw"
sw_val_bpe_output = "/content/val_bpe/val.bpe.sw"

ar_test_input = "/content/test/test.ar"
ar_test_bpe_output = "/content/test_bpe/test.bpe.ar"
sw_test_input = "/content/test/test.sw"
sw_test_bpe_output = "/content/test_bpe/test.bpe.sw"

vocab_size = 12000


sw_source_file = "/content/Swahili.txt"
ar_source_file = "/content/Arabic.txt"

# joint_vocab_file = "/content/joint_train.txt"




#using entire text to found the dictonary for the BPE
def train_ar(vocab_size):
  model_prefix = dict_path+"/ar_" + "_vocab_" + str(vocab_size)
  spm.SentencePieceTrainer.train(input=ar_source_file
      , model_prefix=model_prefix
      , vocab_size=vocab_size
      , character_coverage = 0.9995
      , num_threads=60
      , model_type = "bpe"
      , train_extremely_large_corpus=True
  )
train_ar(vocab_size)

def train_sw(vocab_size):
  model_prefix = dict_path + "/swahil_" + "_vocab_" + str(vocab_size)
  spm.SentencePieceTrainer.train(input=sw_source_file
      , model_prefix=model_prefix
      , vocab_size=vocab_size
      , character_coverage = 0.9995
      , num_threads=60
      ,model_type = "bpe"
      , train_extremely_large_corpus=True
  )
train_sw(vocab_size)

ar_tokenizer = spm.SentencePieceProcessor(model_file="/content/bpe_dict_path/ar__vocab_12000.model")
sw_tokenizer = spm.SentencePieceProcessor(model_file="/content/bpe_dict_path/swahil__vocab_12000.model")


#from the above the BPE models are trained
####################################


#lines uses the Arabic BPE model to tokenize the Arabic training data
with open(ar_train_input, "r", encoding="utf-8") as rf, open(ar_train_bpe_output, "w", encoding="utf-8") as wf:
    output_lines = []
    for line in rf.readlines():
        wf.write(' '.join(ar_tokenizer.encode(line, out_type=str)))
        wf.write("\n")


with open(sw_train_input, "r", encoding="utf-8") as rf, open(sw_train_bpe_output, "w", encoding="utf-8") as wf:
    output_lines = []
    for line in rf.readlines():
        wf.write(' '.join(sw_tokenizer.encode(line, out_type=str)))
        wf.write("\n")


with open(ar_test_input, "r", encoding="utf-8") as rf, open(ar_test_bpe_output, "w", encoding="utf-8") as wf:
    output_lines = []
    for line in rf.readlines():
        wf.write(' '.join(ar_tokenizer.encode(line, out_type=str)))
        wf.write("\n")



with open(sw_test_input, "r", encoding="utf-8") as rf, open(sw_test_bpe_output, "w", encoding="utf-8") as wf:
    output_lines = []
    for line in rf.readlines():
        wf.write(' '.join(sw_tokenizer.encode(line, out_type=str)))
        wf.write("\n")




with open(ar_val_input, "r", encoding="utf-8") as rf, open(ar_val_bpe_output, "w", encoding="utf-8") as wf:
    output_lines = []
    for line in rf.readlines():
        wf.write(' '.join(ar_tokenizer.encode(line, out_type=str)))
        wf.write("\n")




with open(sw_val_input, "r", encoding="utf-8") as rf, open(sw_val_bpe_output, "w", encoding="utf-8") as wf:
    output_lines = []
    for line in rf.readlines():
        wf.write(' '.join(sw_tokenizer.encode(line, out_type=str)))
        wf.write("\n")



In [9]:
!cut -f1 "/content/bpe_dict_path/ar__vocab_12000.vocab" | tail -n +4 | sed "s/$/ 100/g" > "/content/bpe_dict_path/fairseq.ar.vocab"
!cut -f1 "/content/bpe_dict_path/swahil__vocab_12000.vocab" | tail -n +4 | sed "s/$/ 100/g" > "/content/bpe_dict_path/fairseq.sw.vocab"

##FAIRSEQ Preprocess "Prepare the data to go into the Transformer"

In [37]:
SOURCE_LANGUAGE="ar"
TARGET_LANGUAGE="sw"
TRAIN_PREF="/content/train_bpe/train.bpe"
VALID_PREF="/content/val_bpe/val.bpe"
TEST_PREF="/content/test_bpe/test.bpe"
DEST_DIR="/content/ar_sw.tokenized.ar-sw"
SRC_THRES=0
TGT_THRES=0
ar_DICT_PATH="/content/bpe_dict_path/fairseq.ar.vocab"
swahili_DICT_PATH="/content/bpe_dict_path/fairseq.sw.vocab"

!fairseq-preprocess    --source-lang $SOURCE_LANGUAGE --target-lang $TARGET_LANGUAGE  \
   --srcdict "/content/bpe_dict_path/fairseq.ar.vocab"    --tgtdict "/content/bpe_dict_path/fairseq.sw.vocab"  --align-suffix align     --trainpref  $TRAIN_PREF        --validpref $VALID_PREF     --testpref $TEST_PREF   --destdir  $DEST_DIR     --thresholdsrc $SRC_THRES     --thresholdtgt $TGT_THRES

2023-12-07 19:15:47.710713: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 19:15:47.710785: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 19:15:47.710828: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 19:15:47.724386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 19:15:50.362199: W tensorflow/c

##FAIRSEQ Training



In [ ]:
DROPOUT=0.1
ATTENTION_DROPOUT=0
ACTIVATION_DROPOUT=0.1
EMBEDDING_SIZE=256
ENC_FFNN=1024
ENCODER_LAYERS=5
ENCODER_ATTENTION_HEADS=8
DECODER_LAYERS=5
DECODER_ATTENTION_HEADS=8
DEC_FFNN=1024
EPOCH=400
BATCH_SIZE=4096
ENCODER_LAYER_DROPOUT=0.1
DECODER_LAYER_DROPOUT=0.1
LABEL_SMOOTHING=0.1
SAVE_DIR="/content/drive/MyDrive/ironside_nmt/checkpoints/"
LABEL_CROSS_ENTROPY="label_smoothed_cross_entropy"
WARMUP_UPDATES=4000
lEARNING_POLICY="inverse_sqrt"
WAND_PROJECT_NAME="LARGE_SCALE_ARSWAHILI_Translation"

!fairseq-train "/content/ar_sw.tokenized.ar-sw" \
    --arch transformer \
    --dropout $DROPOUT \
    --attention-dropout 0     --encoder-embed-dim $EMBEDDING_SIZE \
    --encoder-ffn-embed-dim $ENC_FFNN  \
    --encoder-layers $ENCODER_LAYERS  \
    --encoder-attention-heads $ENCODER_ATTENTION_HEADS \
    --encoder-learned-pos  \
    --decoder-embed-dim $EMBEDDING_SIZE \
    --decoder-ffn-embed-dim $DEC_FFNN  \
    --decoder-layers $DECODER_LAYERS \
    --decoder-attention-heads $DECODER_ATTENTION_HEADS \
    --decoder-learned-pos   \
    --max-epoch $EPOCH  \
    --optimizer adam \
    --lr 5e-4 \
    --max-tokens $BATCH_SIZE \
    --seed 1     --encoder-layerdrop $ENCODER_LAYER_DROPOUT     --decoder-layerdrop $DECODER_LAYER_DROPOUT \
    --criterion $LABEL_CROSS_ENTROPY     --warmup-updates $WARMUP_UPDATES \
    --source-lang ar    --label-smoothing $LABEL_SMOOTHING \
    --lr-scheduler $lEARNING_POLICY   --save-dir $SAVE_DIR \
    --find-unused-parameters  \
    --target-lang sw \
    --activation-dropout $ACTIVATION_DROPOUT  \
    --ddp-backend=no_c10d \
    --no-epoch-checkpoints --wandb-project $WAND_PROJECT_NAME \
    --log-format=json --log-interval=10 2>&1    |  tee  "/content/drive/MyDrive/logs/training_log.log"

##FAIRSEQ Generating BLEU score (Testing)


In [38]:
BATCH_SIZE=128
BEAM=5
SEED=1
SCORING="bleu"
CHECKPOINT_PATH="/content/drive/MyDrive/ironside_nmt/checkpoints/checkpoint_last.pt"
    # --remove-bpe 'sentencepiece' \

!fairseq-generate "/content/ar_sw.tokenized.ar-sw"\
    --batch-size $BATCH_SIZE \
    --beam $BEAM \
    --path $CHECKPOINT_PATH \
    --remove-bpe 'sentencepiece' \
    --seed $SEED \
    --scoring bleu > "/content/results.txt"

2023-12-07 19:23:01.979698: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 19:23:01.979767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 19:23:01.979812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 19:23:01.994258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 19:23:03.566048: W tensorflow/c

In [21]:
with open( "/content/results.txt" ) as f:
  text = f.read()
  text = text.split()
  print(" ".join(text[-8:-5]))
  # BLEU4 acheived 11.25, after 150 epochs


BLEU4 = 11.71,
